In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
#import duckdb as dd
import polars as pl
import os
import glob
import shutil
import zipfile
import pickle
from joblib import dump, load
from pathlib import Path

In [2]:
random_forest_model = load('/kaggle/input/trained-audio-model-v-01/audio_classifier_model.joblib')

In [3]:
meta_data = pl.read_csv('/kaggle/input/birdclef-2024/train_metadata.csv', low_memory=True)
list(meta_data.columns)

['primary_label',
 'secondary_labels',
 'type',
 'latitude',
 'longitude',
 'scientific_name',
 'common_name',
 'author',
 'license',
 'rating',
 'url',
 'filename']

In [4]:
bird_cols = list(meta_data['primary_label'].unique().sort())

In [5]:
test_soundscapes = '/kaggle/input/birdclef-2024/test_soundscapes'

for path in Path(test_soundscapes).glob("*.ogg"):
    print(path)
    print(path.stem)
    print(path.stem.split("_"))

In [6]:
test = pd.DataFrame(
     [(path.stem, *path.stem.split("_"), path) for path in Path(test_soundscapes).glob("*.ogg")],
    columns = ["filename", "name" ,"id", "path"]
)
print(test.shape)
test.head()

(0, 4)


,filename,name,id,path


In [7]:
filenames = test.filename.values.tolist()

submission_df = pd.DataFrame(columns=['row_id']+bird_cols)
submission_df

,row_id,asbfly,ashdro1,ashpri1,ashwoo2,asikoe2,asiope1,aspfly1,aspswi1,barfly1,...,whbwoo2,whcbar1,whiter2,whrmun,whtkin2,woosan,wynlau1,yebbab1,yebbul3,zitcis1


In [8]:
for i, file in enumerate(filenames):
    predicted = random_forest_model.predict_proba[i]
    num_rows = len(predicted)
    row_ids = [f'{file}_{(i+1)*5}' for i in range(num_rows)]
    df = pd.DataFrame(columns=['row_id']+bird_cols)
    
    df['row_id'] = row_ids
    df[bird_cols] = predicted
    
    submission_df = pd.concat([submission_df,df]).reset_index(drop=True)
    submission_df

In [9]:
submission_df.to_csv('submission.csv', index=False)